In [16]:
import tensorflow as tf
import numpy as np

# 12 텐서플로를 사용한 사용자 정의 모델과 훈련

## 12.1 텐서플로 훑어보기

텐서플로가 지원하는것들

* 핵심 구조는 넘파이와 매우 비슷하지만 GPU를 지원합니다.
* 분산 컴퓨팅을 지원합니다.
* 일종의 JIT컴파일러를 포함합니다. 속도를 높이고 메모리 사용량을 줄이기 위해 계산을 최적화 합니다. 이를 위해 파이썬 함수에서 계산 그래프를 추출한 다음 최적화하고 효율적으로 실행합니다.
* 계산 그래프는 플랫폼에 중립적인 포멧으로 내보낼 수 있으므로 한 환경에서 텐서플로 모델을 훈련하고 다른 환경에서 실행할 수 있습니다.
* 텐서플로는 자동 미분 기능과 RMSProp, Nadam 같은 고성능 옵티마이저를 제공하므로 모든 종류의 손실 함수를 쉽게 최소화 할 수 있습니다.



## 12.2 넘파이처럼 텐서플로 사용하기

텐서플로 API는 텐서를 순환시킵니다. 텐서는 한 연산에서 다른 연산으로 흐릅니다. 그래서 텐서플로라고 부릅니다.  
이제 텐서를 만들고 조작하는 방법을 알아봅시다.

### 12.2.1 텐서와 연산

tf.constant() 함수로 텐서를 만들 수 있습니다.

In [9]:
t = tf.constant([[1.,2.,3.], [4., 5., 6.]]) # 행렬

print(t)
print(t.shape)
print(t.dtype)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
(2, 3)
<dtype: 'float32'>


In [5]:
tf.constant(42) # 스칼라

<tf.Tensor: id=3, shape=(), dtype=int32, numpy=42>

인덱스 참조도 넘파이와 매우 비슷하게 작동합니다.

In [12]:
print(t[:, 1:])
print(t[..., 1, tf.newaxis])

tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[2.]
 [5.]], shape=(2, 1), dtype=float32)


가장 중요한 것은 모든 종류의 텐서 연산이 가능하다는 것입니다.

In [15]:
print(t + 10)
print(tf.square(t))
print(t @ tf.transpose(t))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [18]:
## 12.2.2 텐서와 넘파이

# 넘파이 배열로 텐서를 만들 수 있고 그 반대도 가능합니다. 

a = np.array([2., 4., 5.])
tf.constant(a)
t.numpy()

<tf.Tensor: id=7, shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [19]:
tf.square(a)

<tf.Tensor: id=38, shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [20]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

In [23]:
### 12.2.3 타입변환
# 텐서플로는 타입을 자동으로 변환시키지 않습니다.  
# 타입변환은 tf.case() 를 이용합니다.

t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: id=44, shape=(), dtype=float32, numpy=42.0>

In [24]:
### 12.2.4 변수
# tf.Tensor는 변경 불가능한 객체입니다. 하지만 tf.Variable 을 이용하면 변수처럼 다룰 수 있습니다.

v = tf.Variable([[1.,2.,3.], [4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [25]:
# tf.Variable 로 선언된 변수는 assign 함수를 이용하여 변수값을 바꿀 수 있습니다.a
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

### 12.2.5 다른 데이터 구조

텐서플로는 다음과 같은 몇 가지 다른 데이터 구조도 지원합니다.

* 희소 텐서 sparse tensor
* 텐서 배열 tensor array
* 래그드 텐서 ragged tensor
* 문자열 텐서 string tensor
* 집합 set
* 큐 queue

## 12.3 사용자 정의 모델과 훈련 알고리즘

### 12.3.1 사용자 정의 손실 함수

텐서플로를 사용하여 사용자 정의 손실함수를 구현해봅시다. 여기서는 huber 를 구현해보겠습니다.

In [26]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)